In [ ]:
import pandas as pd
import numpy as np

merged_df = pd.read_pickle("merged.pkl")
vitf_spac = merged_df[(merged_df["model"] == "ViT-Finetuned") & (merged_df["dataset"] == "SPAC")]
df = vitf_spac

In [ ]:
from gorillatracker.classification.metrics import analyse_embedding_space, format_metrics, formatted_names

metrics = analyse_embedding_space(df)
format_metrics(metrics)

In [ ]:
def create_latex_table(metrics, formatted_names):
    latex_table = r"""
\begin{table}
    \centering
    \resizebox{0.8\paperwidth}{!}{
    \begin{tabular}{l *{8}{c}}
    \toprule
    & \multicolumn{2}{c}{Bristol} & \multicolumn{2}{c}{SPAC} & \multicolumn{2}{c}{Bristol} & \multicolumn{2}{c}{SPAC} \\
    \cmidrule(lr){2-3} \cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9}
    Metric & ViT-F & ViT-P & ViT-F & ViT-P & EfN-F & EfN-P & EfN-F & EfN-P \\
    \midrule
"""

    for metric, formatted_name in formatted_names.items():
        row = f"{formatted_name} "
        for model_prefix in ["ViT-", "EfN-"]:
            for dataset in ["Bristol", "SPAC"]:
                for model_suffix in ["F", "P"]:
                    model = f"{model_prefix}{model_suffix}"
                    value = metrics.get((model, dataset), {}).get(metric, "-")
                    if isinstance(value, float):
                        row += f"& {value:.2f} "
                    else:
                        row += f"& {value} "
        row += r"\\"
        latex_table += row + "\n"

    latex_table += r"""    \bottomrule
    \end{tabular}
    }
    \caption{Metrics comparison across models and datasets.}
    \label{tab:metrics-comparison}
\end{table}
"""
    return latex_table


df = merged_df
actual_metrics = {
    ("ViT-F", "SPAC"): analyse_embedding_space(df[(df["model"] == "ViT-Finetuned") & (df["dataset"] == "SPAC")]),
    ("ViT-P", "SPAC"): analyse_embedding_space(df[(df["model"] == "ViT-Pretrained") & (df["dataset"] == "SPAC")]),
    # ('EfN-F', 'SPAC'): analyse_embedding_space(df[(df['model'] == 'EfN-Finetuned') & (df['dataset'] == 'SPAC')]),
    ("EfN-P", "SPAC"): analyse_embedding_space(df[(df["model"] == "EfN-Pretrained") & (df["dataset"] == "SPAC")]),
    ("ViT-F", "Bristol"): analyse_embedding_space(df[(df["model"] == "ViT-Finetuned") & (df["dataset"] == "Bristol")]),
    ("ViT-P", "Bristol"): analyse_embedding_space(df[(df["model"] == "ViT-Pretrained") & (df["dataset"] == "Bristol")]),
    # ('EfN-F', 'Bristol'): analyse_embedding_space(df[(df['model'] == 'EfN-Finetuned') & (df['dataset'] == 'Bristol')]),
    ("EfN-P", "Bristol"): analyse_embedding_space(df[(df["model"] == "EfN-Pretrained") & (df["dataset"] == "Bristol")]),
}

print(create_latex_table(actual_metrics, formatted_names))